In [1]:
# 100000 chunks are created in this example.
import dask.array as da
arrayshape = (200000, 200000)
chunksize = (2000, 2000)
x = da.ones(arrayshape, chunks=chunksize)
x
#x.compute()

dask.array<ones_like, shape=(200000, 200000), dtype=float64, chunksize=(2000, 2000), chunktype=numpy.ndarray>

In [2]:
from dask.diagnostics import ProgressBar
big_calc = (x * x[::-1, ::-1]).sum()
with ProgressBar():
    result = big_calc.compute()
print(f"Total size: {result}")

[########################################] | 100% Completed | 42.8s
Total size: 40000000000.0


In [1]:
# Restart kernel
# Take shorter duration to complete with smaller number (400) of chunks.

import dask.array as da
arrayshape = (200000, 200000)
chunksize = (10000, 10000)
x = da.ones(arrayshape, chunks=chunksize)
x

dask.array<ones_like, shape=(200000, 200000), dtype=float64, chunksize=(10000, 10000), chunktype=numpy.ndarray>

In [2]:
from dask.diagnostics import ProgressBar
big_calc = (x * x[::-1, ::-1]).sum()
with ProgressBar():
    result = big_calc.compute()
print(f"Total size: {result}")

[########################################] | 100% Completed | 26.9s
Total size: 40000000000.0


In [1]:
# Restart kernel
# Allow automatic assignment of chunks. The system assigns 2500 tasks in this example.

import dask.array as da
arrayshape = (200000, 200000)
x = da.ones(arrayshape)
x

dask.array<ones_like, shape=(200000, 200000), dtype=float64, chunksize=(4000, 4000), chunktype=numpy.ndarray>

In [2]:
from dask.diagnostics import ProgressBar
big_calc = (x * x[::-1, ::-1]).sum()
with ProgressBar():
    result = big_calc.compute()
print(f"Total size: {result}")

[########################################] | 100% Completed | 35.2s
Total size: 40000000000.0


In [1]:
# Restart kernel
# Insufficient memory results in kernel crash when assigning higher size of chunk
import dask.array as da
arrayshape = (200000, 200000)
chunksize = (20000, 20000)
x = da.ones(arrayshape, chunks=chunksize)
x

dask.array<ones_like, shape=(200000, 200000), dtype=float64, chunksize=(20000, 20000), chunktype=numpy.ndarray>

In [ ]:
from dask.diagnostics import ProgressBar
big_calc = (x * x[::-1, ::-1]).sum()
with ProgressBar():
    result = big_calc.compute()
print(f"Total size: {result}")

[###                                     ] | 9% Completed |  1.0s

In [1]:
# Restart kernel
# Allow automatic assignment of chunks. The system assigns 2500 tasks in this example.
import dask.array as da
arrayshape = (200000, 200000)
x = da.ones(arrayshape)
x

dask.array<ones_like, shape=(200000, 200000), dtype=float64, chunksize=(4000, 4000), chunktype=numpy.ndarray>

In [2]:
# Automatic assignment of chunks by the system allows computation to complete without encountering insufficient memory problem.
from dask.diagnostics import ProgressBar
big_calc = (x * x[::-1, ::-1]).sum()
with ProgressBar():
    result = big_calc.compute()
print(f"Total size: {result}")

[########################################] | 100% Completed | 35.8s
Total size: 40000000000.0


In [1]:
# Restart kernel. Create a Dask scheduler pod.
import os
import time
import cdsw
import dask

dask_scheduler = cdsw.launch_workers(
    n=1,
    cpu=2,
    memory=2,
    code=f"!dask-scheduler --host 0.0.0.0 --dashboard-address 127.0.0.1:8090",
)

# Wait for the scheduler to start.
time.sleep(10)

In [2]:
# Obtain the Dask UI address.
print("//".join(dask_scheduler[0]["app_url"].split("//")))

http://zusu97y7pconcrlc.ml-76cf996d-8f4.apps.apps.ocp4.cdpkvm.cldr/


In [3]:
# Take note of the Dask scheduler URL.
scheduler_workers = cdsw.list_workers()
scheduler_id = dask_scheduler[0]["id"]
scheduler_ip = [
    worker["ip_address"] for worker in scheduler_workers if worker["id"] == scheduler_id
][0]

scheduler_url = f"tcp://{scheduler_ip}:8786"
scheduler_url

'tcp://10.254.0.98:8786'

In [4]:
from dask.distributed import Client
client = Client(scheduler_url)
client

<Client: 'tcp://10.254.0.39:8786' processes=1 threads=16, memory=14.81 GiB>

In [5]:
import dask.array as da
arrayshape = (200000, 200000)
x = da.ones(arrayshape)
x

dask.array<ones_like, shape=(200000, 200000), dtype=float64, chunksize=(4000, 4000), chunktype=numpy.ndarray>

In [6]:
%%time
big_calc = (x * x[::-1, ::-1]).sum()
result = big_calc.compute()
print(f"Total size: {result}")

Total size: 40000000000.0
CPU times: user 2.17 s, sys: 641 ms, total: 2.81 s
Wall time: 9min 3s


In [7]:
import dask.array as da
arrayshape = (200000, 200000)
chunksize = (10000, 10000)
x = da.ones(arrayshape, chunks=chunksize)
x

dask.array<ones_like, shape=(200000, 200000), dtype=float64, chunksize=(10000, 10000), chunktype=numpy.ndarray>

In [8]:
%%time
big_calc = (x * x[::-1, ::-1]).sum()
result = big_calc.compute()
print(f"Total size: {result}")

Total size: 40000000000.0
CPU times: user 685 ms, sys: 253 ms, total: 937 ms
Wall time: 3min 55s


In [9]:
# Assigning scheduler "single-threaded" doesn't trigger Dask-worker?
%time big_calc = (x * x[::-1, ::-1]).sum().compute(scheduler='single-threaded')
print(f"Total size: {big_calc}")

CPU times: user 1min 23s, sys: 1min 54s, total: 3min 18s
Wall time: 3min 17s
Total size: 40000000000.0


In [10]:
%%time
big_calc = (x * x[::-1, ::-1]).sum().compute(scheduler='threads')
print(f"Total size: {big_calc}")

Total size: 40000000000.0
CPU times: user 3min 2s, sys: 2min 47s, total: 5min 50s
Wall time: 26.9 s


In [1]:
# Restart the kernel. Creata a new Dask cluster.

import os
import time
import cdsw
import dask

dask_scheduler = cdsw.launch_workers(
    n=1,
    cpu=2,
    memory=2,
    code=f"!dask-scheduler --host 0.0.0.0 --dashboard-address 127.0.0.1:8090",
)

# Wait for the scheduler to start.
time.sleep(10)

In [2]:
# Obtain the Dask UI address.
print("//".join(dask_scheduler[0]["app_url"].split("//")))

http://agfeuy4lgg17kf6f.ml-76cf996d-8f4.apps.apps.ocp4.cdpkvm.cldr/


In [3]:
scheduler_workers = cdsw.list_workers()
scheduler_id = dask_scheduler[0]["id"]
scheduler_ip = [
    worker["ip_address"] for worker in scheduler_workers if worker["id"] == scheduler_id
][0]

scheduler_url = f"tcp://{scheduler_ip}:8786"
scheduler_url

'tcp://10.254.0.116:8786'

In [4]:
# Assign 3 worker nodes to the Dask cluster.
more_worker = 3
dask_workers = cdsw.launch_workers(
    n=more_worker,
    cpu=2,
    memory=32,
    code=f"!dask-worker {scheduler_url}",
)

# Wait for the workers to start.
time.sleep(10)

In [5]:
from dask.distributed import Client
client = Client(scheduler_url)
client

<Client: 'tcp://10.254.0.116:8786' processes=3 threads=72, memory=89.13 GiB>

In [6]:
import dask.array as da
arrayshape = (200000, 200000)
chunksize = (10000, 10000)
x = da.ones(arrayshape, chunks=chunksize)
x

dask.array<ones_like, shape=(200000, 200000), dtype=float64, chunksize=(10000, 10000), chunktype=numpy.ndarray>

In [7]:
%%time
big_calc = (x * x[::-1, ::-1]).sum()
result = big_calc.compute()
print(f"Total size: {result}")

Total size: 40000000000.0
CPU times: user 329 ms, sys: 113 ms, total: 443 ms
Wall time: 22.9 s


In [1]:
# Restart kernel. Use GPU to compute.
import cupy as cpcupy
import dask.array as dacupy

arrayshape = (200000, 200000)
chunksize = (10000, 10000)
y = dacupy.ones_like(cpcupy.array(()), shape=arrayshape, chunks=chunksize)
y

dask.array<ones_like, shape=(200000, 200000), dtype=float64, chunksize=(10000, 10000), chunktype=cupy.ndarray>

In [2]:
%time array_sumcupy = dacupy.sum(y).compute()
print(f"Total size: {array_sumcupy}")

CPU times: user 1.09 s, sys: 2.34 s, total: 3.43 s
Wall time: 2.5 s
Total size: 40000000000.0
